<a href="https://colab.research.google.com/github/MukeshSK75/Deep-Learning-Practice/blob/main/Langchain_based_ExpertSystem(heatstroke_risk).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from flask import Flask, render_template, request
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
from langchain_groq import ChatGroq
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

app = Flask(__name__)

# -------------------- LangChain Setup --------------------
llm = ChatGroq(groq_api_key= myown-api-key , model_name="llama3-8b-8192")
conversation = ConversationChain(llm=llm, memory=ConversationBufferMemory())

# -------------------- Fuzzy Variables Setup --------------------
temperature = ctrl.Antecedent(np.arange(0, 51, 1), 'temperature')
humidity = ctrl.Antecedent(np.arange(0, 101, 1), 'humidity')
symptoms = ctrl.Antecedent(np.arange(0, 3, 1), 'symptoms')
risk = ctrl.Consequent(np.arange(0, 101, 1), 'risk')

temperature['low'] = fuzz.trimf(temperature.universe, [0, 0, 25])
temperature['medium'] = fuzz.trimf(temperature.universe, [20, 30, 40])
temperature['high'] = fuzz.trimf(temperature.universe, [35, 50, 50])

humidity['low'] = fuzz.trimf(humidity.universe, [0, 0, 40])
humidity['medium'] = fuzz.trimf(humidity.universe, [30, 50, 70])
humidity['high'] = fuzz.trimf(humidity.universe, [60, 100, 100])

symptoms['none'] = fuzz.trimf(symptoms.universe, [0, 0, 1])
symptoms['mild'] = fuzz.trimf(symptoms.universe, [0, 1, 2])
symptoms['severe'] = fuzz.trimf(symptoms.universe, [1, 2, 2])

risk['low'] = fuzz.trimf(risk.universe, [0, 0, 40])
risk['medium'] = fuzz.trimf(risk.universe, [30, 50, 70])
risk['high'] = fuzz.trimf(risk.universe, [60, 100, 100])

# -------------------- Fuzzy Rules --------------------
rule1 = ctrl.Rule(temperature['low'] & humidity['low'] & symptoms['none'], risk['low'])
rule2 = ctrl.Rule(temperature['low'] & humidity['low'] & symptoms['mild'], risk['low'])
rule3 = ctrl.Rule(temperature['low'] & humidity['low'] & symptoms['severe'], risk['medium'])

rule4 = ctrl.Rule(temperature['low'] & humidity['medium'] & symptoms['none'], risk['low'])
rule5 = ctrl.Rule(temperature['low'] & humidity['medium'] & symptoms['mild'], risk['medium'])
rule6 = ctrl.Rule(temperature['low'] & humidity['medium'] & symptoms['severe'], risk['medium'])

rule7 = ctrl.Rule(temperature['low'] & humidity['high'] & symptoms['none'], risk['medium'])
rule8 = ctrl.Rule(temperature['low'] & humidity['high'] & symptoms['mild'], risk['medium'])
rule9 = ctrl.Rule(temperature['low'] & humidity['high'] & symptoms['severe'], risk['high'])


# --- Group 2: Medium Temperature ---
rule10 = ctrl.Rule(temperature['medium'] & humidity['low'] & symptoms['none'], risk['low'])
rule11 = ctrl.Rule(temperature['medium'] & humidity['low'] & symptoms['mild'], risk['medium'])
rule12 = ctrl.Rule(temperature['medium'] & humidity['low'] & symptoms['severe'], risk['high'])

rule13 = ctrl.Rule(temperature['medium'] & humidity['medium'] & symptoms['none'], risk['low'])
rule14 = ctrl.Rule(temperature['medium'] & humidity['medium'] & symptoms['mild'], risk['medium'])
rule15 = ctrl.Rule(temperature['medium'] & humidity['medium'] & symptoms['severe'], risk['high'])

rule16 = ctrl.Rule(temperature['medium'] & humidity['high'] & symptoms['none'], risk['medium'])
rule17 = ctrl.Rule(temperature['medium'] & humidity['high'] & symptoms['mild'], risk['high'])
rule18 = ctrl.Rule(temperature['medium'] & humidity['high'] & symptoms['severe'], risk['high'])


# --- Group 3: High Temperature ---
rule19 = ctrl.Rule(temperature['high'] & humidity['low'] & symptoms['none'], risk['medium'])
rule20 = ctrl.Rule(temperature['high'] & humidity['low'] & symptoms['mild'], risk['medium'])
rule21 = ctrl.Rule(temperature['high'] & humidity['low'] & symptoms['severe'], risk['high'])

rule22 = ctrl.Rule(temperature['high'] & humidity['medium'] & symptoms['none'], risk['medium'])
rule23 = ctrl.Rule(temperature['high'] & humidity['medium'] & symptoms['mild'], risk['high'])
rule24 = ctrl.Rule(temperature['high'] & humidity['medium'] & symptoms['severe'], risk['high'])

rule25 = ctrl.Rule(temperature['high'] & humidity['high'] & symptoms['none'], risk['high'])
rule26 = ctrl.Rule(temperature['high'] & humidity['high'] & symptoms['mild'], risk['high'])
rule27 = ctrl.Rule(temperature['high'] & humidity['high'] & symptoms['severe'], risk['high'])


# Create the control system with all 27 rules
risk_ctrl = ctrl.ControlSystem([
    rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8, rule9,
    rule10, rule11, rule12, rule13, rule14, rule15, rule16, rule17, rule18,
    rule19, rule20, rule21, rule22, rule23, rule24, rule25, rule26, rule27
])
# Moved simulation object creation into function to avoid reuse bug

# -------------------- LLM-Based Symptom Extraction --------------------
def get_symptom_level_with_llm(symptom_input):
    prompt = f"""You are a medical assistant. Analyze the following patient message and classify the symptoms into one of these levels:
0 = No symptoms,
1 = Mild symptoms (e.g., headache, tired, thirsty, sweating),
2 = Severe symptoms (e.g., nausea, dizziness, fainting, vomiting)

Patient said: "{symptom_input}"

Just respond with a number: 0, 1, or 2. Nothing else."""
    try:
        response = conversation.predict(input=prompt)
        level = int(''.join(filter(str.isdigit, response)))
        return min(max(level, 0), 2)
    except:
        return 0  # Fallback to none if LLM fails

# -------------------- Risk Calculation --------------------
def assess_risk(temp, hum, symptom_level):
    risk_simulator = ctrl.ControlSystemSimulation(risk_ctrl)  # New simulator each time
    risk_simulator.input['temperature'] = temp
    risk_simulator.input['humidity'] = hum
    risk_simulator.input['symptoms'] = symptom_level
    risk_simulator.compute()
    return risk_simulator.output['risk']

# -------------------- Flask Routes --------------------
@app.route('/')
def index():
    return render_template('index.html')

@app.route('/home', methods=['POST'])
def home():
    temperature_input = float(request.form['temperature'])
    humidity_input = float(request.form['humidity'])
    symptoms_input = request.form['symptoms']

    # Get fuzzy value from LLM
    symptom_level = get_symptom_level_with_llm(symptoms_input)
    risk_level = assess_risk(temperature_input, humidity_input, symptom_level)

    return render_template('result.html', risk=risk_level)

# -------------------- Run App --------------------
if __name__ == '__main__':
    app.run(debug=True)
